In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


In [3]:
excel_file = r"vi_prod_class_22-10.xlsx"  # Đường dẫn đến file Excel của bạn

In [4]:
df_1 = pd.read_excel(excel_file, sheet_name = r'tranformed Kqxd trước mã số', skiprows=1).iloc[:,:3].fillna('')
df_2 = pd.read_excel(excel_file, sheet_name = r'transformed Phân loại hàng hóa ', skiprows=1).iloc[:,:3].fillna('')

df = pd.concat([df_1,df_2])
df.columns = ['link','doc_no','doc_date']
df = df.drop_duplicates()
df['doc_name'] = df['doc_no'].replace('/', '_', regex=True).replace('-', '_', regex=True)+'_'+df['doc_date'].replace('-', '_', regex=True)


In [5]:
df.head()

,link,doc_no,doc_date,doc_name
0,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3664/TB-TCHQ,29-07-2024,3664_TB_TCHQ_29_07_2024
1,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3349/TB-TCHQ,10-07-2024,3349_TB_TCHQ_10_07_2024
2,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3313/TB-TCHQ,08-07-2024,3313_TB_TCHQ_08_07_2024
3,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,3312/TB-TCHQ,08-07-2024,3312_TB_TCHQ_08_07_2024
4,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,2710/TB-TCHQ,12-06-2024,2710_TB_TCHQ_12_06_2024


In [6]:
# df.to_excel("link_for_download_pdf.xlsx",index = True)

In [26]:
# Đường dẫn để đăng nhập
login_url = 'https://thuvienphapluat.vn/page/ajaxcontroler.aspx'

# Dữ liệu cần gửi để đăng nhập
payload = {
    'l_txtUser': 'UNICUSVN',
    'l_txtPass': 'UNIUNI',
    'action': 'Login'
}

# Headers cần thiết cho yêu cầu
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Origin': 'https://thuvienphapluat.vn',
    'Pragma': 'no-cache',
    'Referer': 'https://thuvienphapluat.vn/page/login.aspx?',
    'Sec-CH-UA': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

# Khởi tạo session để giữ cookie
session = requests.Session()

# Gửi yêu cầu POST để đăng nhập
login_response = session.post(login_url, data=payload, headers=headers)
# Kiểm tra đăng nhập thành công
if login_response.ok:
    print("Đăng nhập thành công!")
else:
    print("Đăng nhập thất bại:", login_response.status_code)


Đăng nhập thành công!


In [8]:

def get_downloadlink(link):
    # Bước 1: Gửi yêu cầu tới link
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        find_pdf_link = soup.find('div', id='tab8').find('a', id='ctl00_Content_ThongTinVB_pdfHyperLink')
        if not find_pdf_link:
            print("Không có link PDF trong div 'tab8'.")
        else:
            try:
                all_script = soup.find_all('script')
                pattern = r'/documents/download\.aspx\?id=[\w%]+'
                if not all_script:
                    print("Không tìm thấy thẻ script nào trong trang.")
                else:
                    for text in all_script:
                        if '__urldl' in text.get_text():
                            matches = re.findall(pattern, text.get_text())
                            if matches:
                                download_url = f'https://thuvienphapluat.vn{matches[-1]}&part=-0'
                                print("Download URL:", download_url)
                                return download_url
            except Exception as e:
                print(f"Lỗi khi tìm các thẻ script và")      
    except Exception as e:
        print(f"Lỗi khi tìm link PDF: {e}")
        

In [24]:
def download_pdf(download_link,file_name,idx):
    # path = r"D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_pdf\\"
    path = r"D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_pdf\test_1\\"
    # Gửi yêu cầu tải file PDF
    response = session.get(download_link, headers=headers, stream=True)

    # Kiểm tra nếu yêu cầu thành công và lưu file
    if response.ok:
        
        with open(f"{path}{file_name}_{idx}.pdf", "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Tải file thành công và lưu thành {file_name}.pdf")
    else:
        print("Lỗi khi tải file:", response.status_code)


In [10]:
# for idx,row in df.iloc[150:170,:].iterrows():
#     print (idx)
#     print(row[0])
#     download_link= get_downloadlink(row[0])
#     if not download_link:
#         print("không có link download")
#     else:
#         print(row[3])
#         file_name = row[3]
#         download_pdf(download_link,file_name,idx)

In [11]:
from PyPDF2 import PdfReader

def is_pdf_valid(file_path):
    try:
        # Mở và kiểm tra file PDF
        with open(file_path, "rb") as f:
            pdf = PdfReader(f)
            # Thử truy cập một thuộc tính của file để kiểm tra tính hợp lệ
            pdf.pages[0]  # Kiểm tra trang đầu tiên
        return True  # File PDF hợp lệ
    except Exception as e:
        print(f"File PDF {file_path} có lỗi: {e}")
        return False  # File PDF không hợp lệ


In [12]:
import concurrent.futures
import os

# Thư mục chứa các tệp tải về
download_folder = r"D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_pdf\\"

# Hàm xử lý một dòng (công việc)
def process_row(idx, row):
    file_name = row[3]
    file_local_name = f"{row[3]}_{idx}.pdf"
    print(file_local_name)
    file_path = os.path.join(download_folder, file_local_name)
    
    # Kiểm tra nếu tệp đã tồn tại ngay khi bắt đầu
    if os.path.exists(file_path) and is_pdf_valid(file_path):
        print(f"{file_local_name} đã tồn tại, bỏ qua tải xuống.")
        return  # Dừng hàm nếu tệp đã có
    else:
        print(idx)
        print(row[0])
        download_link = get_downloadlink(row[0])
        
        if not download_link:
            print("không có link download")
        else:
            print(f"Tải xuống {file_local_name} ...")
            try:
                download_pdf(download_link, file_name, idx)
                print(f"{file_local_name} đã được tải xuống thành công.")
            except Exception as e:
                print(f"Lỗi khi tải xuống {file_local_name}: {e}")


In [ ]:
# Số lượng luồng
num_threads = 10

# Tạo danh sách công việc để xử lý trong các luồng
rows_to_process = [(idx, row) for idx, row in df.iloc[170:].iterrows()]

# Sử dụng ThreadPoolExecutor để tạo multithreading
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(lambda args: process_row(*args), rows_to_process)

C:\Users\nguye\AppData\Local\Temp\ipykernel_22552\2203351599.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  file_name = row[3]
C:\Users\nguye\AppData\Local\Temp\ipykernel_22552\2203351599.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  file_local_name = f"{row[3]}_{idx}.pdf"
C:\Users\nguye\AppData\Local\Temp\ipykernel_22552\2203351599.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])
C:\Users\nguye\AppData

4361_TB_TCHQ_21_08_2023_170.pdf
4360_TB_TCHQ_21_08_2023_171.pdf
4341_TB_TCHQ_18_08_2023_172.pdf
4288_TB_TCHQ_17_08_2023_173.pdf
4287_TB_TCHQ_17_08_2023_174.pdf
4284_TB_TCHQ_17_08_2023_175.pdf
4361_TB_TCHQ_21_08_2023_170.pdf đã tồn tại, bỏ qua tải xuống.
4283_TB_TCHQ_17_08_2023_176.pdf
3333_TB_TCHQ_11_08_2022_177.pdf
3332_TB_TCHQ_11_08_2022_178.pdf
4141_TB_TCHQ_08_08_2023_179.pdf
4111_TB_TCHQ_07_08_2023_180.pdf
4288_TB_TCHQ_17_08_2023_173.pdf đã tồn tại, bỏ qua tải xuống.
4130_TB_TCHQ_07_08_2023_181.pdf
4341_TB_TCHQ_18_08_2023_172.pdf đã tồn tại, bỏ qua tải xuống.
4129_TB_TCHQ_07_08_2023_182.pdf
4360_TB_TCHQ_21_08_2023_171.pdf đã tồn tại, bỏ qua tải xuống.
4127_TB_TCHQ_07_08_2023_183.pdf
4283_TB_TCHQ_17_08_2023_176.pdf đã tồn tại, bỏ qua tải xuống.
4110_TB_TCHQ_07_08_2023_184.pdf
4284_TB_TCHQ_17_08_2023_175.pdf đã tồn tại, bỏ qua tải xuống.
4066_TB_TCHQ_03_08_2023_185.pdf
4287_TB_TCHQ_17_08_2023_174.pdf đã tồn tại, bỏ qua tải xuống.
4065_TB_TCHQ_03_08_2023_186.pdf
3333_TB_TCHQ_11_08_202

In [27]:
download_pdf('https://thuvienphapluat.vn/documents/download.aspx?id=8j8vviS1FqrbQ209WXD1Kg%3d%3d&part=-0','test_name',10)

Tải file thành công và lưu thành test_name.pdf
